# Tutorial: QUA calibration nodes on a graph

This tutorial explains, with a simple toy example, how to run calibrations on a graph using the *QUA Calibration node*
framework. It is based on the code in [hello QUA](https://github.com/qua-platform/qua-libs/tree/main/examples/basics/hello-qua)
in the [qua-libs](https://github.com/qua-platform/qua-libs) project.

The basic idea of the QUA calibration nodes is to allow the quantum engineer or experimentalist to build up the
quantum machine config required to run an accurate and updated QUA program in an iterative way, and to break it down
to a manageable series of small and self-contained steps, each within a `QuaCalNode`.

## Prerequisites and assumptions

This tutorial assumes a working knowledge of QUA (see [qua-libs](https://github.com/qua-platform/qua-libs))
and familiarity with the QPU DB
(see [section 1](https://github.com/entropy-lab/entropy-qpu/blob/main/docs/qpu_db.ipynb) of the tutorial).

## QUA Calibration nodes

todo: insert diagram explaining the idea

## initializing a QPU DB and an entropy DB

The following code, similar to [section 1](https://github.com/entropy-lab/entropy-qpu/blob/main/docs/qpu_db.ipynb)
of the tutorial, creates a QPU DB. It also creates an entropy DB and registers the QPU DB as a resource.

In [7]:
%load_ext autoreload
%autoreload 2
from entropylab_qpudb import create_new_qpu_database, CalState, QpuDatabaseConnection
from entropylab.instruments.lab_topology import LabResources, ExperimentResources
from entropylab.results_backend.sqlalchemy.db import SqlAlchemyDB

initial_dict = {
    'q1': {
        'f01': 5.65e9,  # an initial guess for our transition frequency
        'pi_pulse_amp': 0.6  # an initial guess for a pi-pulse amplitude
    },
}
create_new_qpu_database('db1', initial_dict, force_create=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


in the following lines we register the resource:

In [8]:
entropydb = SqlAlchemyDB('entropy_db.db')
lab_resources = LabResources(entropydb)
lab_resources.register_resource_if_not_exist(
    'qpu_db', QpuDatabaseConnection, ['db1']
)

Now we have the databases ready to go and to be used.

## creating the root node

In [9]:
from entropylab import EntropyContext, pynode
from qm.qua import *

from entropylab_qpudb import QuaConfig


@pynode("root", output_vars={"config"})
def root(context: EntropyContext):
    f_lo_1 = 5.5e9  # this is a parameter that we control and so we don't place it in the QPU DB
    return {"config": QuaConfig({
        "version": 1,
        "controllers": {
            "con1": {
                "type": "opx1",
                "analog_outputs": {
                    1: {"offset": +0.0},
                },
            }
        },
        "elements": {
            "q1": {
                "singleInput": {"port": ("con1", 1)},
                "intermediate_frequency": context.get_resource('qpu_db').q(1).f01.value - f_lo_1,
                "operations": {
                    "playOp": "constPulse",
                },
            },
        }
    })}

# Measuring $f_{01}$ and updating the QPU DB and config

todo: explain about the classes

In [10]:
from entropylab_qpudb import QuaCalNode

class QubitSpecNode(QuaCalNode):
    def prepare_config(self, config: QuaConfig, context: EntropyContext):
        # todo: add a measurement pulse
        pass

    def run_program(self, config, context: EntropyContext):
        # todo: simple program and update QPU DB
        pass

    def update_config(self, config: QuaConfig, context: EntropyContext):
        pass

class PowerRabiNode(QuaCalNode):

    def prepare_config(self, config: QuaConfig, context: EntropyContext):
        pass

    def run_program(self, config, context: EntropyContext):
        pass

    def update_config(self, config: QuaConfig, context: EntropyContext):
        pass

## Prepare graph experiment

In [11]:
qubit_spec_q1 = QubitSpecNode(dependency=root)
power_rabi_q1 = PowerRabiNode(dependency=qubit_spec_q1)

from entropylab import Graph
experiment_resources = ExperimentResources(entropydb)
experiment_resources.import_lab_resource('qpu_db')
calibration_experiment = Graph(experiment_resources, power_rabi_q1.ancestors())
calibration_experiment.run()

opening qpu database db1 from commit <timestamp: 05/27/2021 09:15:23, message: initial commit> at index 0
2021-05-27 12:15:23,719 - entropy - INFO - Running node <PyNode> root
2021-05-27 12:15:23,723 - entropy - INFO - Running node <QubitSpecNode> QubitSpecNode
2021-05-27 12:15:23,729 - entropy - INFO - Running node <PowerRabiNode> PowerRabiNode
2021-05-27 12:15:23,738 - entropy - INFO - Finished entropy experiment execution successfully


In [12]:
experiment_resources.get_resource('qpu_db').close()

closing qpu database db1
